In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My Drive/Thesis_Resources/DynamicWorldV1

/gdrive/My Drive/Thesis_Resources/DynamicWorldV1


In [ ]:
import ee
import geemap
import os

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=D0I_fam_SbH_5yAx15IMlA_9M0D8tQQNN7unvruuCLY&tc=abdVrOkwCSPZePpMYyMfik3ONcxz7xSY7G_3zbVuDk0&cc=7DCkWKXZ5S-n-7AtGC4TJZFGj66rQwHVP_DsSFSYMW0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXk4QhKJ0D0JDCdHExgiUXjs4pLmoX8rqpO-vNW5MvsIR7XwiIExrvA

Successfully saved authorization token.


In [ ]:
# Initialize the Earth Engine module.
ee.Initialize()

In [36]:
import ee
import geemap

class DynamicWorldVisualizer:
    """
    A class to visualize Dynamic World and Sentinel-2 datasets.

    Attributes:
        start_date (ee.Date): The start date for the data collection.
        end_date (ee.Date): The end date for the data collection.
        location (ee.Geometry.Point): The geographical point of interest.
        map_object (geemap.Map): The map object for visualization.
    """

    CLASS_NAMES = [
        'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
        'shrub_and_scrub', 'built', 'bare', 'snow_and_ice',
    ]

    VIS_PALETTE = [
        '419bdf', '397d49', '88b053', '7a87c6', 'e49635',
        'dfc35a', 'c4281b', 'a59b8f', 'b39fe1',
    ]

    def __init__(self, start_date, end_date, longitude, latitude):
        """
        Initializes the DynamicWorldVisualizer with specified parameters.

        Args:
            start_date (str): The start date in 'YYYY-MM-DD' format.
            end_date (str): The end date in 'YYYY-MM-DD' format.
            longitude (float): The longitude of the point of interest.
            latitude (float): The latitude of the point of interest.
        """
        self.start_date = ee.Date(start_date)
        self.end_date = ee.Date(end_date)
        self.location = ee.Geometry.Point(longitude, latitude)
        self.col_filter = ee.Filter.And(
            ee.Filter.bounds(self.location),
            ee.Filter.date(self.start_date, self.end_date),
        )
        self.map_object = geemap.Map()


    def _collect_data(self):
        """
        Collects and joins the Dynamic World and Sentinel-2 datasets.
        """

        dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(self.col_filter)
        s2_col = ee.ImageCollection('COPERNICUS/S2').filter(self.col_filter)

        return ee.Join.saveFirst('s2_img').apply(
            dw_col, s2_col, ee.Filter.equals(leftField='system:index', rightField='system:index')
        )


    def _create_visualization(self, dw_s2_col):
        """
        Creates a visualization blending Dynamic World class labels with probabilities.

        Args:
            dw_s2_col (ee.ImageCollection): The collection of joined DW and S2 images.
        """
        dw_image = ee.Image(dw_s2_col.first())
        s2_image = ee.Image(dw_image.get('s2_img'))

        dw_rgb = (
            dw_image.select('label')
            .visualize(min=0, max=8, palette=self.VIS_PALETTE)
            .divide(255)
        )

        top1_prob = dw_image.select(self.CLASS_NAMES).reduce(ee.Reducer.max())
        top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)
        dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

        self.map_object.add_layer(
            s2_image,
            {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
            'Sentinel-2 L1C'
        )
        self.map_object.add_layer(
            dw_rgb_hillshade,
            {'min': 0, 'max': 0.65},
            'Dynamic World V1 - label hillshade'
        )


    def _add_legend(self):
        """
        Adds a legend to the map for LULC classes.
        """
        legend_title = 'LULC Classes'
        legend_colors = ['#' + color for color in self.VIS_PALETTE]
        legend_dict = dict(zip(self.CLASS_NAMES, legend_colors))
        self.map_object.add_legend(legend_title=legend_title, legend_dict=legend_dict, position='bottomright')


    def display_map(self):
        """
        Displays the map with LULC data and legend.
        """
        dw_s2_col = self._collect_data()
        self._create_visualization(dw_s2_col)
        self._add_legend()
        self.map_object.set_center(self.location.coordinates().get(0).getInfo(), self.location.coordinates().get(1).getInfo(), 12)
        return self.map_object

In [34]:
# Usage example
visualizer = DynamicWorldVisualizer('2023-04-02', '2023-12-03', 9.18951, 45.46427)
visualizer.display_map()

Map(center=[45.46427, 9.18951], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [37]:
visualizer.get_rgb_image_url()

'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/54261b3bad884ffca025ea1c3daa7033-2610f51580fc2ccea98bad4fabf43558:getPixels'